In [3]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/PC3_Termination3/')

Mounted at /content/drive


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler
from joblib import Parallel, delayed
import joblib
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [5]:
model_saving_path = '/content/drive/MyDrive/PC3_Termination3/MC_checkpoints/'

## Model

In [6]:
# 4 layered model

# Set a random seed for NumPy for CPU operations
seed = 42
np.random.seed(seed)

# Set a random seed for PyTorch for GPU (if available) and CPU operations
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Define a custom neural network with dropout
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(32*32, 128)
#         self.fc2 = nn.Linear(128, 128)
#         self.fc3 = nn.Linear(128, 64)
#         self.fc4 = nn.Linear(64, 10)  # Output layer
#         self.dropout = nn.Dropout(p=0.2)  # Dropout layer with a dropout probability of 0.2

#     def forward(self, x):
#         x = x.view(-1, 32*32)  # Flatten the input
#         x = torch.relu(self.fc1(x))
#         x = self.dropout(x)  # Apply dropout
#         x = torch.relu(self.fc2(x))
#         x = self.dropout(x)  # Apply dropout
#         x = torch.relu(self.fc3(x))
#         x = self.dropout(x)  # Apply dropout
#         x = self.fc4(x)  # Output layer
#         return F.softmax(x, dim=1)  # Apply softmax activation

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(64*64, 256)  # Increase the input size and hidden units
        self.fc2 = nn.Linear(256, 128)    # Adjust the hidden layer sizes
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)      # Output layer
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = x.view(-1, 64*64)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return F.softmax(x, dim=1)

# Initialize the network
net = Net()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)  # lr = lr / gamma after every step_size number of epochs

# Load the MNIST dataset
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])   # convert data to tensors and transform it to have mean 0.5 and varience 0.5
# transform = transforms.Compose([transforms.ToTensor()])   # convert data to tensors
# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
batch_size = 64
transform = transforms.Compose([transforms.Resize(64), transforms.ToTensor()])
train_data = datasets.MNIST(root='./MNIST_dataset', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./MNIST_dataset', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [7]:
# Training
num_epoch = 0
# Training loop
for epoch in range(num_epoch):  # You can increase the number of epochs
    running_loss = 0.0
    running_loss_per_epoch = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_loss_per_epoch += loss.item()
        if i % 100 == 99:  # Print loss every 100 mini-batches
            print(f'Epoch {epoch + 1}, Mini-Batch {i + 1}, Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    joblib.dump(net, f'{model_saving_path}/MC_epoch_{epoch+1}.pkl')
    print(f'Epoch {epoch + 1}, Loss: {running_loss_per_epoch / 100:.4f}')
    # Update the learning rate
    scheduler.step()
    # Access the current learning rate from the optimizer
    current_lr = optimizer.param_groups[0]['lr']

    # Print the current learning rate
    print(f"Current Learning Rate: {current_lr}")


print('Finished Training')

Finished Training


In [8]:
loading_path = f'{model_saving_path}MC_epoch_10.pkl'
net = joblib.load(loading_path)

## Evaluation

In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)  # Assuming `net` is your trained model

        # Get the index of the maximum output (predicted class)
        _, predicted = torch.max(outputs.data, 1)

        # Update the counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print('Accuracy on the test set: {}%'.format(accuracy))

Accuracy on the test set: 96.77%


## MC Dropout

In [20]:
# class CustomDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.png')]

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_name = os.path.join(self.root_dir, self.image_files[idx])
#         image = Image.open(img_name)
#         if self.transform:
#             image = self.transform(image)
#         return image

# # loading images from file
# generated_images_file_path = '/content/drive/MyDrive/PC3_Termination3/New_WGAN_output_images/'
# generated_transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),])
# generated_dataset = CustomDataset(root_dir=generated_images_file_path, transform=generated_transform)
# generated_batch_size = 64  # Adjust as needed
# generated_dataloader = DataLoader(generated_dataset, batch_size=generated_batch_size, shuffle=False)

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return idx, image  # Include the index (batch number) in the returned tuple

# Loading images from file
epoch_num = 2
generated_images_file_path = f'/content/drive/MyDrive/PC3_Termination3/New_WGAN_epochwise_images/{epoch_num}'
generated_transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),])
generated_dataset = CustomDataset(root_dir=generated_images_file_path, transform=generated_transform)
generated_batch_size = 10000  # = number of images generated per epoch
generated_dataloader = DataLoader(generated_dataset, batch_size=generated_batch_size, shuffle=False)

In [21]:
# To implement MC Dropout for uncertainty estimation, sample predictions multiple times with dropout enabled.

# Set a random seed for NumPy for CPU operations
seed = 42
np.random.seed(seed)

# Set a random seed for PyTorch for GPU (if available) and CPU operations
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

net.train()  # Set the network to training mode
num_samples = 50  # Number of samples for MC Dropout
stacked_tensor = torch.tensor([])
# image_count = 0

with torch.no_grad():
  # for inputs in generated_dataloader:
  # for batch_number, (idx, inputs) in generated_dataloader:
  for batch_number, (idx, inputs) in enumerate(generated_dataloader):
    # image_count += generated_batch_size
    # if image_count == 300:
    if batch_number == epoch_num-1:
      print("batch_number: ", batch_number)
      print("index: ", idx)
      predcitions_tensor = torch.tensor([])
      for i in range(num_samples):
          outputs = net(inputs)     # output.shape = torch.Size([batch_size, 10])
          predcitions_tensor = torch.cat((predcitions_tensor, outputs.unsqueeze(2)), dim=2)
      print("stacked_tensor_temp.shape: ", predcitions_tensor.shape)
      stacked_tensor = torch.cat((stacked_tensor, predcitions_tensor), dim=0)
      print("stacked_tensor.shape: ", stacked_tensor.shape)

In [22]:
for batch_number, (idx, inputs) in enumerate(generated_dataloader):
  print(idx)

tensor([   0,    1,    2,  ..., 4811, 4812, 4813])


In [14]:
# Calculate mean and variance of the predictions
# predictions = torch.stack(predictions, dim=0)  # Stack predictions along a new dimension, predictions.shape = [num_samples x batch_size x 10], type(predictions) = torch.Tensor
mean_prediction = torch.mean(stacked_tensor, dim=2)  # mean_prediction.shape = torch.Size([batch_size, 10])
variance_prediction = torch.var(stacked_tensor, dim=2) # variance_prediction.shape = torch.Size([batch_size, 10])

In [15]:
mean1 = torch.mean(mean_prediction)
varience1 = torch.var(variance_prediction)
print(mean1)
print(varience1)

tensor(0.1000)
tensor(0.0014)


In [17]:
# Find the maximum values and their indices along dim=1 (columns)
max_values_mean, _ = torch.max(mean_prediction, dim=1)

# Find the index of the maximum mean value along dim=1 (columns)
max_mean_indices = torch.argmax(mean_prediction, dim=1)

# Use the max_mean_indices to extract corresponding variance values
max_variance_values = torch.gather(variance_prediction, dim=1, index=max_mean_indices.unsqueeze(1)).squeeze(1)

print("max_values_mean: ", max_values_mean)
print("max_mean_indices: ", max_mean_indices)
print("max_variance_values: ", max_variance_values)

max_values_mean:  tensor([0.9999, 0.9716, 0.9866,  ..., 0.9526, 0.7362, 0.8137])
max_mean_indices:  tensor([3, 3, 3,  ..., 2, 3, 8])
max_variance_values:  tensor([6.8288e-07, 1.4385e-02, 8.7862e-03,  ..., 3.3099e-02, 5.7250e-02,
        5.6536e-02])


In [19]:
print(max_values_mean.mean())
print(max_variance_values.mean())

tensor(0.8236)
tensor(0.0670)


In [ ]:
'''
On all digits - correct and incorrect
mean1
variance1

Accuracy on the test set: 96.88%

For epoch1 and 2 combined
tensor(0.1000)
tensor(0.0013)


Epoch1:
tensor(0.1000)
tensor(0.0015)

Epoch2:
tensor(0.1000)
tensor(0.0011)
'''

In [ ]:
'''
Accuracy on the test set: 96.88%

For epoch1 and 2 combined
tensor(0.8259)
tensor(0.0656)

Epoch1:
tensor(0.7768)
tensor(0.0841)

Epoch2:
tensor(0.8691)
tensor(0.0495)
'''

In [ ]:
# old
'''
Mean and varience of true preds only:
Generated image epoch 30:
tensor(0.6512)
tensor(0.1272)

Generated image epoch 50:
tensor(0.6730)
tensor(0.1215)

Generated image epoch 95:
tensor(0.6295)
tensor(0.1355)
'''

'\nMean and varience of true preds only:\nGenerated image epoch 30:\ntensor(0.6512)\ntensor(0.1272)\n\nGenerated image epoch 50:\n\n\nGenerated image epoch 95:\n\n'

In [ ]:
# old
'''
Mean and varience of all:
Generated image epoch 30:
tensor(0.1000)
tensor(0.0024)

Generated image epoch 50:
tensor(0.1000)
tensor(0.0022)

Generated image epoch 95:
tensor(0.1000)
tensor(0.0029)
'''

In [ ]:
'''
Epoch = 1
mean1 = tensor(-1.7371)
varience1 = tensor(0.7028)

Softmax
tensor(0.1000)
tensor(0.0001)


Epoch = 3
Accuracy on the test set: 95.91%
tensor(-2.5869)
tensor(3.4694)

Softmax
Accuracy on the test set: 94.72%
tensor(0.1000)
tensor(8.9567e-05)



Epoch = 5
Accuracy on the test set: 96.29%
tensor(-3.4590)
tensor(11.5309)

Softmax
Accuracy on the test set: 95.62%
tensor(0.1000)
tensor(1.2447e-05)


Epoch = 10
Accuracy on the test set: 96.46%
Epoch 10, Loss: 14.0068
tensor(0.1000)
tensor(1.2360e-06)


One extra layer
Epoch = 30
Accuracy on the test set: 96.64%
Epoch 30, Loss: 13.9579
tensor(0.1000)
tensor(4.9995e-05)
'''

'\nEpoch = 1\nmean1 = tensor(-1.7371)\nvarience1 = tensor(0.7028)\n\nSoftmax\ntensor(0.1000)\ntensor(0.0001)\n\n\nEpoch = 3\nAccuracy on the test set: 95.91%\ntensor(-2.5869)\ntensor(3.4694)\n\nSoftmax\nAccuracy on the test set: 94.72%\ntensor(0.1000)\ntensor(8.9567e-05)\n\n\n\nEpocch = 5\nAccuracy on the test set: 96.29%\ntensor(-3.4590)\ntensor(11.5309)\n\nSoftmax\nAccuracy on the test set: 95.62%\ntensor(0.1000)\ntensor(1.2447e-05)\n\n\nEpoch = 10\nAccuracy on the test set: 96.46%\nEpoch 10, Loss: 14.0068\ntensor(0.1000)\ntensor(1.2360e-06)\n\n\nEpoch = 30\nAccuracy on the test set: 96.64%\n\n'

## MC Dropout Old

In [ ]:
# Now you can use the trained network to make predictions.
# To implement MC Dropout for uncertainty estimation, you can sample predictions multiple times with dropout enabled.
# Here's an example of making predictions with MC Dropout:

# Set a random seed for NumPy for CPU operations
seed = 42
np.random.seed(seed)

# Set a random seed for PyTorch for GPU (if available) and CPU operations
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

net.train()  # Set the network to training mode
num_samples = 100  # Number of samples for MC Dropout
predictions = []

with torch.no_grad():
    for _ in range(num_samples):
        outputs = net(inputs)     # output.shape = torch.Size([batch_size, 10])
        predictions.append(outputs)

In [ ]:
# Calculate mean and variance of the predictions
predictions = torch.stack(predictions, dim=0)  # Stack predictions along a new dimension, predictions.shape = [num_samples x batch_size x 10], type(predictions) = torch.Tensor
mean_prediction = torch.mean(predictions, dim=0)  # mean_prediction.shape = torch.Size([batch_size, 10])
variance_prediction = torch.var(predictions, dim=0) # variance_prediction.shape = torch.Size([batch_size, 10])

In [ ]:
mean1 = torch.mean(mean_prediction)
varience1 = torch.var(variance_prediction)
print(mean1)
print(varience1)

tensor(0.1000)
tensor(3.9906e-06)


## Old

In [ ]:
# 4 layered model

# Set a random seed for NumPy for CPU operations
seed = 42
np.random.seed(seed)

# Set a random seed for PyTorch for GPU (if available) and CPU operations
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Define a custom neural network with dropout
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)  # Output layer
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with a dropout probability of 0.2

    def forward(self, x):
        x = x.view(-1, 32*32)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc4(x)  # Output layer
        return F.softmax(x, dim=1)  # Apply softmax activation

# Initialize the network
net = Net()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)  # lr = lr / gamma after every step_size number of epochs

# Load the MNIST dataset
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])   # convert data to tensors and transform it to have mean 0.5 and varience 0.5
# transform = transforms.Compose([transforms.ToTensor()])   # convert data to tensors
# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
batch_size = 64
transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor()])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

num_epoch = 30
# Training loop
for epoch in range(num_epoch):  # You can increase the number of epochs
    running_loss = 0.0
    running_loss_per_epoch = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_loss_per_epoch += loss.item()
        if i % 100 == 99:  # Print loss every 100 mini-batches
            print(f'Epoch {epoch + 1}, Mini-Batch {i + 1}, Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    joblib.dump(net, f'{model_saving_path}/net_epoch_{epoch+1}.pkl')
    print(f'Epoch {epoch + 1}, Loss: {running_loss_per_epoch / 100:.4f}')
    # Update the learning rate
    scheduler.step()
    # Access the current learning rate from the optimizer
    current_lr = optimizer.param_groups[0]['lr']

    # Print the current learning rate
    print(f"Current Learning Rate: {current_lr}")


print('Finished Training')

In [ ]:
# 3 layered model
# Set a random seed for NumPy for CPU operations
seed = 42
np.random.seed(seed)

# Set a random seed for PyTorch for GPU (if available) and CPU operations
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Define a custom neural network with dropout
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # Output layer
        self.dropout = nn.Dropout(p=0.2)  # Dropout layer with a dropout probability of 0.2

    def forward(self, x):
        x = x.view(-1, 32*32)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc3(x)  # Output layer
        return F.softmax(x, dim=1)  # Apply softmax activation

# Initialize the network
net = Net()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)  # lr = lr / gamma after every step_size number of epochs

# Load the MNIST dataset
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])   # convert data to tensors and transform it to have mean 0.5 and varience 0.5
# transform = transforms.Compose([transforms.ToTensor()])   # convert data to tensors
# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
batch_size = 64
transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor()])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

num_epoch = 10
# Training loop
for epoch in range(num_epoch):  # You can increase the number of epochs
    running_loss = 0.0
    running_loss_per_epoch = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_loss_per_epoch += loss.item()
        if i % 100 == 99:  # Print loss every 100 mini-batches
            print(f'Epoch {epoch + 1}, Mini-Batch {i + 1}, Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    joblib.dump(net, f'{model_saving_path}/net_epoch_{epoch+1}.pkl')
    print(f'Epoch {epoch + 1}, Loss: {running_loss_per_epoch / 100:.4f}')
    # Update the learning rate
    scheduler.step()
    # Access the current learning rate from the optimizer
    current_lr = optimizer.param_groups[0]['lr']

    # Print the current learning rate
    print(f"Current Learning Rate: {current_lr}")


print('Finished Training')

In [ ]:
# Older version

# Now you can use the trained network to make predictions.
# To implement MC Dropout for uncertainty estimation, you can sample predictions multiple times with dropout enabled.
# Here's an example of making predictions with MC Dropout:

# Set a random seed for NumPy for CPU operations
seed = 42
np.random.seed(seed)

# Set a random seed for PyTorch for GPU (if available) and CPU operations
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

net.train()  # Set the network to training mode
num_samples = 50  # Number of samples for MC Dropout

# predictions = []

# with torch.no_grad():
#   for inputs in generated_dataloader:
#     print(inputs.shape)               # print(inputs.shape) = torch.Size([batch_size, 1, 32, 32])
#     for i in range(num_samples):
#         outputs = net(inputs)     # output.shape = torch.Size([batch_size, 10])
#         print(outputs.shape)
#         predictions.append(outputs)

# # predcitions_tensor = torch.tensor([])
# stacked_tensor = torch.tensor([])

# with torch.no_grad():
#   for inputs in generated_dataloader:
#     # print(inputs.shape)               # print(inputs.shape) = torch.Size([batch_size, 1, 32, 32])
#     # for i in range(num_samples):
#     predictions_lst = []
#     for i in range(num_samples):
#         outputs = net(inputs)     # output.shape = torch.Size([batch_size, 10])
#         # print(outputs.shape)
#         predictions_lst.append(outputs)
#     stacked_tensor_temp = torch.stack(predictions_lst, dim=2)
#     print("stacked_tensor_temp.shape", stacked_tensor_temp.shape)
#     # if stacked_tensor is None:
#     #     stacked_tensor = stacked_tensor_temp
#     # else:
#     #     # Stack the individual tensor on top of the existing stacked_tensor along dimension 0
#     #     stacked_tensor = torch.cat((stacked_tensor, stacked_tensor_temp), dim=0)

#     stacked_tensor = torch.cat((stacked_tensor, stacked_tensor_temp), dim=0)
#     print(stacked_tensor.shape)



stacked_tensor = torch.tensor([])

with torch.no_grad():
  for inputs in generated_dataloader:
    predcitions_tensor = torch.tensor([])
    for i in range(num_samples):
        outputs = net(inputs)     # output.shape = torch.Size([batch_size, 10])
        predcitions_tensor = torch.cat((predcitions_tensor, outputs.unsqueeze(2)), dim=2)
    print("stacked_tensor_temp.shape", predcitions_tensor.shape)
    stacked_tensor = torch.cat((stacked_tensor, predcitions_tensor), dim=0)
    print(stacked_tensor.shape)

In [ ]:
# # Define a custom neural network with dropout
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(28*28, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 10)  # Output layer
#         self.dropout = nn.Dropout(p=0.5)  # Dropout layer with a dropout probability of 0.5

#     def forward(self, x):
#         x = x.view(-1, 28*28)  # Flatten the input
#         x = torch.relu(self.fc1(x))
#         x = self.dropout(x)  # Apply dropout
#         x = torch.relu(self.fc2(x))
#         x = self.dropout(x)  # Apply dropout
#         x = self.fc3(x)  # Output layer
#         return x

# # Initialize the network
# net = Net()

# # Define loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# # Load the MNIST dataset
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# # Training loop
# for epoch in range(5):  # You can increase the number of epochs
#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         inputs, labels = data
#         optimizer.zero_grad()

#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         if i % 100 == 99:  # Print every 100 mini-batches
#             print(f'Epoch {epoch + 1}, Mini-Batch {i + 1}, Loss: {running_loss / 100:.4f}')
#             running_loss = 0.0

# print('Finished Training')

# # Now you can use the trained network to make predictions.
# # To implement MC Dropout for uncertainty estimation, you can sample predictions multiple times with dropout enabled.
# # Here's an example of making predictions with MC Dropout:

# # Now, let's test the model on the MNIST test dataset.
# net.eval()  # Set the network to evaluation mode

# testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         inputs, labels = data
#         outputs = net(inputs)
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(f'Accuracy on test dataset: {100 * correct / total:.2f}%')

In [ ]:
# # Define a CNN model
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
#         self.fc1 = nn.Linear(64*5*5, 128)
#         self.fc2 = nn.Linear(128, 10)

#     # def forward(self, x):
#     #     x = torch.relu(self.conv1(x))
#     #     x = torch.relu(self.conv2(x))
#     #     x = x.view(-1, 64*5*5)
#     #     x = torch.relu(self.fc1(x))
#     #     x = self.fc2(x)
#     #     return x
#     # def forward(self, x):
#     #     x = torch.relu(self.conv1(x))
#     #     x = torch.relu(self.conv2(x))
#     #     # Calculate the correct size based on the output feature map dimensions
#     #     x = x.view(x.size(0), -1)
#     #     x = torch.relu(self.fc1(x))
#     #     x = self.fc2(x)
#     #     return x
#     def forward(self, x):
#         x = torch.relu(self.conv1(x))
#         x = torch.relu(self.conv2(x))

#         # Flatten the tensor, assuming correct output size from conv2
#         x = x.view(x.size(0), -1)

#         x = torch.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# # Load the MNIST dataset and apply transformations
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# # Initialize the model and optimizer
# model = Net()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Train the model
# for epoch in range(5):
#     for images, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

# # Load the test dataset
# test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# # Evaluate the model on test data
# correct = 0
# total = 0
# predictions = []
# confidence_interval = []

# with torch.no_grad():
#     for images, labels in test_loader:
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         confidence = torch.softmax(outputs, dim=1)
#         max_confidence = torch.max(confidence, dim=1)
#         predictions.extend(predicted.numpy())
#         confidence_interval.extend(max_confidence.values.numpy())

# print(f"Test Accuracy: {100 * correct / total:.2f}%")

# # Display results for a specific test example (e.g., index 0)
# print(f"Predicted Digit: {predictions[0]}")
# print(f"Confidence Interval: {confidence_interval[0]:.2f}")
